In [ ]:
import pandas as pd
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
filedir = '<dataset folder here>'

In [ ]:
#df = pd.read_csv('data/landsat8_data_with_fire.csv')

df = pd.read_csv(os.path.join(filedir, 'dataset_path40_39_with_fire_v1.csv'))
                

df=df.drop(columns=["Unnamed: 0","Unnamed: 0.1"])
df.head(2)

In [ ]:
df_fire = df
df_fire['fire_nofire'] = df_fire['fire_nofire'].map(lambda x : 'no_fire' if x == 'no fire' else 'fire')
df_fire.head(2)
df = df_fire

In [ ]:
df_fire = df_fire[(df_fire.width == 512) & (df_fire.height == 512)]

In [ ]:
df_dataset = df_fire[df_fire.fire_nofire == 'fire'].append(df_fire[df_fire.fire_nofire == 'no_fire'][0:2000])
#df_dataset = df_fire

In [ ]:
df_dataset.shape

In [ ]:
from PIL import TiffImagePlugin

def resize(image, width, height):
    #print(image)
    
    #im1 = Image.open(image)
    
    width = width
    height = height
    # use one of these filter options to resize the image
    im2 = image.resize((width, height), Image.NEAREST)      # use nearest neighbour
    return im2

In [ ]:
def plotgrid(dataframe):
    images=[]
    classes=[]
    for item in df_date.itertuples():
        #print(item.filename)
        filename, ext = os.path.splitext(item.filename)
        rowcol = filename.split("_")
        loc = "{}:{}".format(rowcol[9], rowcol[10])
        filename = os.path.join(filedir, 'ndvijpegs',filename+".jpeg")
    
        #print(filename)
        im = Image.open(filename)
        
        im = resize(im, 100, 100)
        images.append(im)
        classes.append(item.fire_nofire)
    fig = plt.figure(figsize=(10, 10))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(16, 16),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )
    #im_fire = Image.open("data/fire.png")
    #im_fire = resize(im_fire, 25,25)
    for ax, im, c in zip(grid, images,classes):
        # Iterating over the grid returns the Axes.
        ax.text(50,50, loc)
        ax.imshow(im,cmap='terrain')
        
        if c == 'fire':
            extent = 0, 100, 0, 100
            kwargs = {"alpha":0.5}
            rect = patches.Rectangle((0,0),100,100,linewidth=1,edgecolor='r',facecolor='r', **kwargs)
            #rect.set_alpha(0.1)
            ax.add_patch(rect)
            #ax.imshow(im_fire, alpha=0.1, extent=extent)

    plt.show()

In [ ]:

Path={}
for item in glob(os.path.join(filedir, 'ndvijpegs', '*.jpeg')):
    #print(item)
    filename = os.path.splitext(os.path.basename(item))[0]
    #print(filename)
    Path[filename+".TIF"] = item
 

In [ ]:
df_dataset['path'] = df_dataset['filename'].map(Path.get)
df_dataset.head(2)

In [ ]:
df_dataset['image'] = df_dataset['path'].map(lambda x: np.asarray(Image.open(x).resize((28,28))))

In [ ]:
df_dataset.head(2)

In [ ]:
df_dataset['image'] = df_dataset['image'].map(lambda x: np.array(x).reshape(x.shape[0],x.shape[1]))
#df_dataset['image'][0].reshape(512,512,1)

In [ ]:
%matplotlib notebook
import matplotlib.animation as animation
fig = plt.figure()
ims = []
for item in df9.itertuples():
    plt.text(20, 20, item.fire_nofire, fontsize=12)
    im = plt.imshow(item.image, cmap='RdYlGn_r', animated=True)
    
    ims.append([im])
    #print(item.fire_nofire)

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,
                                repeat_delay=1000)

ani.save('dynamic_images3.mp4')

plt.show()

In [ ]:
%matplotlib notebook
fig = plt.figure()


def f(x, y):
    return np.sin(x) + np.cos(y)

x = np.linspace(0, 2 * np.pi, 120)
y = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)

im = plt.imshow(df_dataset['image'][1000], animated=True)

i = 1000
#def updatefig(*args):
def updatefig(*args):
    global i
    global df_dataset
    print(i)
    i+=1
    im.set_array(df_dataset['image'][i])
    return im,

ani = animation.FuncAnimation(fig, updatefig, interval=10, blit=True)
plt.show()

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

TWOPI = 2*np.pi

fig, ax = plt.subplots()

t = np.arange(0.0, TWOPI, 0.001)
s = np.sin(t)
l = plt.plot(t, s)

ax = plt.axis([0,TWOPI,-1,1])

redDot, = plt.plot([0], [np.sin(0)], 'ro')

def animate(i):
    redDot.set_data(i, np.sin(i))
    return redDot,

# create animation using the animate() function
myAnimation = animation.FuncAnimation(fig, animate, frames=np.arange(0.0, TWOPI, 0.1), \
                                      interval=10, blit=True, repeat=True)

plt.show()


In [ ]:
#df_dataset = df_dataset[df_dataset.width == 512]

In [ ]:
X=df_dataset.drop(columns=["fire_nofire"])
Y=df_dataset["fire_nofire"]
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
     X, Y, test_size=0.33, random_state=42)

#X_train = np.array(X_train).reshape(10078, 28,28,1)


In [ ]:
x_train = np.asarray(X_train['image'].tolist())

x_test = np.asarray(X_test['image'].tolist())
#x_train = x_train.reshape(x_train.shape[0], *(512,512,1))
#x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))

xTrainmean = np.mean(x_train)
xTrainstd = np.std(x_train)
xTestmean = np.mean(x_test)
xTeststd = np.std(x_test)
x_train = (x_train - xTrainmean)/xTrainstd
x_test = (x_test - xTestmean)/xTeststd

In [ ]:
import keras

In [ ]:
#import sys
#!{sys.executable}  -m pip install --user keras

In [ ]:
#!{sys.executable}  -m pip install --user tensorflow

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

t1 = le.fit_transform(Y_train)
t2 = le.fit_transform(Y_test)
y_train = keras.utils.to_categorical(t1, num_classes=2)
y_test = keras.utils.to_categorical(t2, num_classes=2)
y_train.shape

In [ ]:
Y_train[0]

In [ ]:
y_train[0]

In [ ]:
x_train.shape

In [ ]:
nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))
d2_train_dataset.shape

In [ ]:
nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))
d2_test_dataset.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM


In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(x_train.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.1))


model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))


In [ ]:
from tensorflow.keras.metrics import AUC
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy', tf.keras.metrics.AUC()],
)

history = model.fit(x_train,
          y_train,
          epochs=100,
          )


In [ ]:
print(model.summary())

In [ ]:
from sklearn.metrics import classification_report
import sklearn.metrics as metrics

#from keras import metrics


y_pred = model.predict(x_test.astype('float64'))
target_names=['fire', 'no_fire']
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=target_names))


matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))


In [ ]:
matrix

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

In [ ]:
#plot loss during training
plt.subplot(211)
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['loss'], label='test')
plt.legend()
# plot accuracy during training
plt.subplot(212)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
#plt.plot(history.history['valaccuracy'], label='test')
plt.legend()


In [ ]:
from sklearn.metrics import roc_curve
y_pred_keras = model.predict(x_test)
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test.argmax(axis=1), y_pred_keras.round().argmax(axis=1))
fpr_keras_train, tpr_keras_train, thresholds_keras_train = roc_curve(y_train.argmax(axis=1), y_train.round().argmax(axis=1))

from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
#plt.plot(fpr_keras_train, tpr_keras_train, label='Keras (area = {:.3f})'.format(auc_keras))
#plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
#plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')
plt.show()

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

conf_matx = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax())
sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap="viridis")
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

# predict probabilities for X_test using predict_proba
probabilities = model.predict(x_test)

# select the probabilities for label 1.0
y_pred = probabilities[:, 1]

# calculate false positive rate and true positive rate at different thresholds
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred, pos_label=1)

# calculate AUC
roc_auc = auc(false_positive_rate, true_positive_rate)

plt.title('Receiver Operating Characteristic')
# plot the false positive rate on the x axis and the true positive rate on the y axis
roc_plot = plt.plot(false_positive_rate,
                    true_positive_rate,
                    label='AUC = {:0.2f}'.format(roc_auc))

plt.legend(loc=0)
plt.plot([0,1], [0,1], ls='--')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate');